## Notebook with SPARQL Kernel

To run this notebook, install the [SPARQL kernel](https://github.com/paulovn/sparql-kernel):
```
pip install sparqlkernel
jupyter sparqlkernel install
```

To use Graphviz to draw graphs, you should also install Pydot and Graphviz:
```
conda install pydot graphviz
```
Set the path to the dot script in your PATH variable. For Miniconda users on Windows, the path would be like `C:\Users\user\Miniconda3\Library\bin\graphviz`

In [1]:
%endpoint https://data-issa.cirad.fr/sparql
%display table
%show 20

%prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
%prefix owl:    <http://www.w3.org/2002/07/owl#>
%prefix xsd:    <http://www.w3.org/2001/XMLSchema#>

%prefix agrovoc: <http://aims.fao.org/aos/agrovoc/>
%prefix dce:    <http://purl.org/dc/elements/1.1/>
%prefix dct:    <http://purl.org/dc/terms/>
%prefix issa:   <http://data-issa.cirad.fr/>
%prefix issapr: <http://data-issa.cirad.fr/property/>
%prefix oa:     <http://www.w3.org/ns/oa#>
%prefix prov:   <http://www.w3.org/ns/prov#>
%prefix schema: <http://schema.org/>
%prefix skos:   <http://www.w3.org/2004/02/skos/core#> 
%prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>

%prefix gn:     <http://www.geonames.org/ontology#>
%prefix geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
%prefix spatial: <http://jena.apache.org/spatial#>
%prefix units:  <http://www.opengis.net/def/uom/OGC/1.0/>

Endpoint set to: https://data-issa.cirad.fr/sparql
Display: table
Result maximum size: 20
Prefix set: rdfs: = <http://www.w3.org/2000/01/rdf-schema#>
Prefix set: owl: = <http://www.w3.org/2002/07/owl#>
Prefix set: xsd: = <http://www.w3.org/2001/XMLSchema#>
Prefix set: agrovoc: = <http://aims.fao.org/aos/agrovoc/>
Prefix set: dce: = <http://purl.org/dc/elements/1.1/>
Prefix set: dct: = <http://purl.org/dc/terms/>
Prefix set: issa: = <http://data-issa.cirad.fr/>
Prefix set: issapr: = <http://data-issa.cirad.fr/property/>
Prefix set: oa: = <http://www.w3.org/ns/oa#>
Prefix set: prov: = <http://www.w3.org/ns/prov#>
Prefix set: schema: = <http://schema.org/>
Prefix set: skos: = <http://www.w3.org/2004/02/skos/core#>
Prefix set: skosxl: = <http://www.w3.org/2008/05/skos-xl#>
Prefix set: gn: = <http://www.geonames.org/ontology#>
Prefix set: geo: = <http://www.w3.org/2003/01/geo/wgs84_pos#>
Prefix set: spatial: = <http://jena.apache.org/spatial#>
Prefix set: units: = <http://www.opengis.net/def/uom/OGC/1.0/>

____

# Geographic named entities

### 1. Get the geographic NEs of a specific article

Look for the geographic named entities of article http://data-issa.cirad.fr/document/455916, and retrieve their details from GeoNames (stored locally in named graph http://geonames.org/graph).

The alternate names of the entity are concatenated in variable ?alternateNames.

In [9]:
SELECT ?geoEntity ?name ?latitude ?longitude ?altitude ?officialName (group_concat(distinct ?alternateName ; separator=", ") as ?alternateNames) ?nameParentCountry
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
WHERE {
    []  a issa:GeographicAnnotation;
        schema:about <http://data-issa.cirad.fr/document/455916>;
        oa:hasBody ?geoEntity.
    
    ?geoEntity
        gn:name   ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
        OPTIONAL {
            ?geoEntity gn:officialName ?officialName
            FILTER langMatches(lang(?officialName), "en")
        }.
        OPTIONAL {
            ?geoEntity gn:alternateName ?alternateName.
            FILTER langMatches(lang(?alternateName), "en")
        }.
        OPTIONAL { ?geoEntity geo:alt ?altitude }.
        OPTIONAL { ?geoEntity gn:parentCountry  [ gn:name ?nameParentCountry ] }.
} group by ?geoEntity ?name ?latitude ?longitude ?altitude ?officialName ?nameParentCountry

geoEntity,name,latitude,longitude,altitude,officialName,alternateNames,nameParentCountry
https://sws.geonames.org/124544/,Māzandarān,36.25,52.33333,,Mazandaran,Māzandarān Province,Iran
https://sws.geonames.org/132892/,Gorgan,36.8427,54.44391,,,Gorgan,Iran
https://sws.geonames.org/130758/,Iran,32,53,,Iran,"Empire of Iran, Iran, Islamic Republic Of, Islamic Republic of Iran",


### 2.1 Find the geographic NEs of documents with descriptor "cacao"

Look for the documents that have the Agrovoc "cacao" concept (http://aims.fao.org/aos/agrovoc/c_7713) as a descriptor.

Then get the geographic named entities of those documents and retrieve their details from GeoNames.

In [4]:
SELECT distinct ?geoEntity ?name ?latitude ?longitude
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?document;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    []  a issa:GeographicAnnotation;
        schema:about ?document;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name         ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
} limit 10

geoEntity,name,latitude,longitude
https://sws.geonames.org/2410758/,São Tomé and Príncipe,1,7
https://sws.geonames.org/6362989/,Les,42.8086,0.71674
https://sws.geonames.org/144739/,Afīn,33.52733,59.7516
https://sws.geonames.org/2640623/,Par,50.35107,-4.70288
https://sws.geonames.org/3666482/,Une,4.30691,-74.07104
https://sws.geonames.org/1733045/,Malaysia,2.5,112.5
https://sws.geonames.org/1253783/,Una,20.82318,71.03795
https://sws.geonames.org/6252001/,United States,39.76,-98.5
https://sws.geonames.org/6255149/,North America,46.07323,-100.54688
https://sws.geonames.org/4338615/,Reserve,30.05381,-90.55175


### 2.2 Find the geographic NEs of documents with descriptor "cacao", with the documents

Similar to the previous query but also return the documents.

In [5]:
SELECT distinct ?document ?documentVisu ?geoEntity ?name ?latitude ?longitude ?altitude
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?document;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    []  a issa:GeographicAnnotation;
        schema:about ?document;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name         ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
    
    ?document dct:identifier ?id.
    bind(iri(concat("http://issa.i3s.unice.fr/visu/?uri=", ?document)) as ?documentVisu)
} limit 10

document,documentVisu,geoEntity,name,latitude,longitude,altitude
http://data-issa.cirad.fr/document/327181,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/327181,https://sws.geonames.org/2410758/,São Tomé and Príncipe,1,7,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/6362989/,Les,42.8086,0.71674,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/144739/,Afīn,33.52733,59.7516,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/2640623/,Par,50.35107,-4.70288,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/3666482/,Une,4.30691,-74.07104,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/1733045/,Malaysia,2.5,112.5,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/1253783/,Una,20.82318,71.03795,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/6252001/,United States,39.76,-98.5,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/6255149/,North America,46.07323,-100.54688,
http://data-issa.cirad.fr/document/388362,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/388362,https://sws.geonames.org/4338615/,Reserve,30.05381,-90.55175,


### 3. Find the documents with geographic NEs within a certain distance of a given point

In [6]:
SELECT distinct ?document ?documentVisu
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
WHERE {
    []  a issa:GeographicAnnotation;
        schema:about ?document;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name   ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
    
    FILTER (bif:st_within(bif:st_point(xsd:float(?longitude), xsd:float(?latitude)), bif:st_point (8, 10), 2000))
    
    ?document dct:identifier ?id.
    bind(iri(concat("http://issa.i3s.unice.fr/visu/?uri=", ?document)) as ?documentVisu)
} LIMIT 10

document,documentVisu
http://data-issa.cirad.fr/document/545102,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/545102
http://data-issa.cirad.fr/document/567662,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/567662
http://data-issa.cirad.fr/document/586054,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/586054
http://data-issa.cirad.fr/document/586500,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/586500
http://data-issa.cirad.fr/document/592959,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/592959
http://data-issa.cirad.fr/document/593766,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/593766
http://data-issa.cirad.fr/document/593897,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/593897
http://data-issa.cirad.fr/document/596414,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/596414
http://data-issa.cirad.fr/document/597271,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/597271
http://data-issa.cirad.fr/document/597792,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/document/597792


____

# Search by descriptors

### 4. Find all documents' descriptors, ordered by number of occurrences (nb of documents that have that descriptor)

In [7]:
%show 50

SELECT distinct ?concept ?label (count(?concept) as ?count)
FROM <http://agrovoc.fao.org/graph>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasBody ?concept.
    ?concept  skosxl:prefLabel [ skosxl:literalForm ?label ].
    
    filter (langMatches(lang(?label), "fr"))
} group by ?concept ?label order by desc(?count)

Result maximum size: 50

concept,label,count
http://aims.fao.org/aos/agrovoc/c_3081,France,3670
http://aims.fao.org/aos/agrovoc/c_1229,Cameroun,1376
http://aims.fao.org/aos/agrovoc/c_4027,Côte d'Ivoire,1291
http://aims.fao.org/aos/agrovoc/c_6970,Sénégal,1134
http://aims.fao.org/aos/agrovoc/c_1070,Brésil,1080
http://aims.fao.org/aos/agrovoc/c_4510,Madagascar,1073
http://aims.fao.org/aos/agrovoc/c_8355,Afrique occidentale,995
http://aims.fao.org/aos/agrovoc/c_806,banane,911
http://aims.fao.org/aos/agrovoc/c_8081,Burkina Faso,887
http://aims.fao.org/aos/agrovoc/c_6543,La Réunion,865


### 5. Find all documents' descriptors in a subtree of concepts, ordered by number of occurrences (nb of documents that have that descriptor)

In [8]:
%show 50

SELECT distinct ?concept ?label (count(?concept) as ?count)
FROM <http://agrovoc.fao.org/graph>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasBody ?concept.

    ?concept  skosxl:prefLabel [ skosxl:literalForm ?label ].
    filter (langMatches(lang(?label), "fr"))
    
    ?concept skos:broader* agrovoc:c_201. # all concepts "politique agricole" or beneath
    
} group by ?concept ?label order by desc(?count)

Result maximum size: 50

concept,label,count
http://aims.fao.org/aos/agrovoc/c_10967,sécurité alimentaire,383
http://aims.fao.org/aos/agrovoc/c_201,politique agricole,375
http://aims.fao.org/aos/agrovoc/c_195,politique foncière,104
http://aims.fao.org/aos/agrovoc/c_4178,réforme foncière,66
http://aims.fao.org/aos/agrovoc/c_196,réforme agraire,22
http://aims.fao.org/aos/agrovoc/c_9000078,sécurité alimentaire des ménages,12
http://aims.fao.org/aos/agrovoc/c_7d40c88b,accès à la nourriture,8
http://aims.fao.org/aos/agrovoc/c_4173,remembrement,2
http://aims.fao.org/aos/agrovoc/c_57ba042c,sécurité nutritionnelle,1
http://aims.fao.org/aos/agrovoc/c_37734,distribution des terres,1


____

# Search by country

### 6. Count documents with descriptor "cacao", that mention places in some given countries

In [10]:
%show 50

SELECT (count(distinct ?document) as ?count) ?countryName
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?document;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    
    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?document.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country { 
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }
    
} order by ?countryName

Result maximum size: 50

count,countryName
61,Brazil
87,Cameroon
23,Costa Rica
28,Ecuador
56,France
53,Ghana


### 7. Count documents with descriptors in some given crop caterogies, that mention places in some given countries

In [11]:
%show 50

SELECT (count(distinct ?document) as ?count) ?cropName ?countryName
FROM <http://agrovoc.fao.org/graph>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?document;
        oa:hasBody ?crop.

    { ?crop skos:broader+ agrovoc:c_1474 } # cereals
    UNION
    { agrovoc:c_7974 <http://aims.fao.org/aos/agrontology#includes> ?crop } # <fruits tropicaux> includes ...
             
    ?crop skosxl:prefLabel [ skosxl:literalForm ?cropName ].
    filter (langMatches(lang(?cropName), "fr"))


    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?document.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country {
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }
    
} order by ?cropName desc(?count)

Result maximum size: 50

count,cropName,countryName
1,Annona,Brazil
1,Annona,Cameroon
1,Annona,Costa Rica
197,agrume,France
98,agrume,Brazil
46,agrume,Cameroon
38,agrume,Ecuador
37,agrume,Costa Rica
25,agrume,Ghana
92,ananas (fruits),France


### 8. Count documents with descriptors in some given crop caterogies, that mention places in some given countries

In [12]:
%show 50

SELECT (count(distinct ?document) as ?count) ?cropName ?countryName
FROM <http://agrovoc.fao.org/graph>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?document;
        oa:hasBody ?crop.

    values ?crop {
        # Tropical fruits
        <http://aims.fao.org/aos/agrovoc/c_13394> # ananas (fruits)
        <http://aims.fao.org/aos/agrovoc/c_9022>  # avocat
        <http://aims.fao.org/aos/agrovoc/c_806>   # banane
        <http://aims.fao.org/aos/agrovoc/c_7713>  # cacao
        <http://aims.fao.org/aos/agrovoc/c_12367> # mangue
        
        # Cereals
        <http://aims.fao.org/aos/agrovoc/c_8373>  # blé
        <http://aims.fao.org/aos/agrovoc/c_12332> # maïs
        <http://aims.fao.org/aos/agrovoc/c_4838>  # millet
        <http://aims.fao.org/aos/agrovoc/c_6599>  # riz
        <http://aims.fao.org/aos/agrovoc/c_7249>  # sorgho en grain
    }

    ?crop skosxl:prefLabel [ skosxl:literalForm ?cropName ].
    filter (langMatches(lang(?cropName), "fr"))


    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?document.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country {
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }
    
} order by ?cropName desc(?count)

Result maximum size: 50

count,cropName,countryName
87,Theobroma cacao,Cameroon
61,Theobroma cacao,Brazil
56,Theobroma cacao,France
53,Theobroma cacao,Ghana
28,Theobroma cacao,Ecuador
23,Theobroma cacao,Costa Rica
92,ananas (fruits),France
66,ananas (fruits),Brazil
49,ananas (fruits),Costa Rica
44,ananas (fruits),Cameroon


### 8b. Same as above but lists all the documents instead of count them

In [13]:
%show 50

SELECT distinct ?document ?title ?date ?abstract ?countryName
FROM <http://agrovoc.fao.org/graph>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/documents>
FROM <http://data-issa.cirad.fr/graph/documents/text>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    # ----- Crops
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?document;
        oa:hasBody ?crop.

    values ?crop {
        # Tropical fruits
        <http://aims.fao.org/aos/agrovoc/c_13394> # ananas (fruits)
        <http://aims.fao.org/aos/agrovoc/c_9022>  # avocat
        <http://aims.fao.org/aos/agrovoc/c_806>   # banane
        <http://aims.fao.org/aos/agrovoc/c_7713>  # cacao
        <http://aims.fao.org/aos/agrovoc/c_12367> # mangue
        
        # Cereals
        <http://aims.fao.org/aos/agrovoc/c_8373>  # blé
        <http://aims.fao.org/aos/agrovoc/c_12332> # maïs
        <http://aims.fao.org/aos/agrovoc/c_4838>  # millet
        <http://aims.fao.org/aos/agrovoc/c_6599>  # riz
        <http://aims.fao.org/aos/agrovoc/c_7249>  # sorgho en grain
    }

    ?crop skosxl:prefLabel [ skosxl:literalForm ?cropName ].
    filter (langMatches(lang(?cropName), "fr"))

    
    # ----- Geographic entities
    ?geoAnnot
        a issa:GeographicAnnotation;
        schema:about ?document.
    
    # Either the geo annotation points to Country or any of its subterritories
    { ?geoAnnot oa:hasBody ?country }
    UNION             
    { ?geoAnnot oa:hasBody [ gn:parentCountry ?country ]. }

    # Considered countries
    ?country gn:name ?countryName.
    values ?country {
        <https://sws.geonames.org/3017382/> # France
        <https://sws.geonames.org/3469034/> # Brazil
        <https://sws.geonames.org/3624060/> # Costa Rica
        <https://sws.geonames.org/3658394/> # Ecuador
        <https://sws.geonames.org/2233387/> # Cameroon
        <https://sws.geonames.org/2300660/> # Ghana
    }


    # ---- documents details
    ?document dct:title ?title ; dct:issued ?date ; dct:abstract [ rdf:value ?abstract ].
    
} order by ?countryName

Result maximum size: 50

document,title,date,abstract,countryName
http://data-issa.cirad.fr/document/437615,Contribution à l'étude de Leucaena leucocephala (LAM.) de WIT. en région tropicale,1987,"Contribution à la connaissance de Leucaena leucocephala (LAM.) de WIT., légumineuse fixatrice d'azote. La croissance, la production de biomasse et la composition minérale de cette plante sont étudiées dans les conditions écologiques de la Martinique, du Brésil et de la Côte d'Ivoire. Ces données permettent d'estimer les quantités de matière végétale qu'il faudrait apporter pour utiliser cette légumineuse comme 'engrais végétal' appliqué sous forme de paillis. Intérêt de Leucaena comme culture de rotation avec le bananier et nécessité d'une étude sur sa sensibilité aux nématodes",Brazil
http://data-issa.cirad.fr/document/510722,"Repères septembre 2002 conjoncture en France : banane, avocat, orange, mangue campagne Afrique de l'Ouest",2002,"Banane : Après avoir connu une reprise rapide en août, le marché s'est à nouveau orienté à la baisse. Le traditionnel développement post-estival des arrivages a été précoce et brutal. Parallèlement, la rentrée n'a pas eu l'effet stimulateur escompté sur la demande locale, d'autant plus que le niveau de prix au détail est resté élevé. Les marchés export ont absorbé de bons volumes, mais sans permettre de remonter un prix moyen décevant. Avocat : L'offre s'est avérée aussi limitée qu'en 2001 et en recul de plus de 10% par rapport à une campagne moyenne. Par ailleurs, la demande s'est développée progressivement. Ainsi, le prix moyen a atteint un niveau historique. Cependant, l'équilibre, très fragile en raison de prix de détail frisant les 1 euro la pièce, s'est rompu en fin de mois entraînant le marché dans une spirale de baisse. Orange : La performance réalisée en septembre est relativement satisfaisante malgré une demande locale irrégulière et versatile. En effet, le prix moyen reste d'un bon niveau même si l'approvisionnement du marché français a été lourd. Le flux export actif vers l'Europe du Nord a joué un rôle important. Mangue campagne Afrique de l'ouest : Après la période difficile de l'été, la marché de la mangue est entré, en septembre, dans une nouvelle phase de transition. L'achèvement progressif de la campagne du Sénégal, la faiblesse des livraisons d'Israël et la diminution sensible des arrivages du Brésil ont permis l'assainissement du marché. Pour autant, la demande reste limitée et le redressement des cours s'effectue lentement.",Brazil
http://data-issa.cirad.fr/document/529609,"Conjoncture France, septembre 2005 : repères. Banane, avocat, orange, pomelo, litchi, mangue, ananas, fret maritime",2005,"Banane: Nouvelle performance exceptionnelle du marché ce mois-ci. Les arrivages globaux sont remontés à un niveau très proche de la moyenne. Cependant, et malgré des sorties plutôt modérées dans l'Hexagone, la situation est restée satisfaisante grâce à une demande active de la clientèle export espagnole, italienne, anglaise et allemande. Ainsi, malgré un fléchissement saisonnier en fin de mois, les prix se sont maintenus à des niveaux jamais vus depuis 1997. Avocat: Comme on le pressentait depuis la deuxième quinzaine du mois d'août, le marché s'est fortement détérioré. Les arrivages ont été massifs. Les exportateurs mexicains se sont concentrés sur le marché des Etats-Unis, mais ceux du Chili ont affirmé leur volonté de prendre une place croissante en Europe à cette époque de l'année. Le prix marque un recul d'environ 30% par rapport à la moyenne triennale, la consommation étant habituellement très calme à cette période de pré-rentrée. Orange: Le mois de septembre n'aura apporté aucune embellie. D'une part, des températures dignes d'un été indien ont pesé sur la consommation d'agrumes en général. D'autre part, les disponibilités ont été supérieures à celles de l'an passé, tant pour l'Afrique du Sud que pour l'Argentine. Le prix moyen mensuel revient au niveau des années 2002 et 2003, après l'embellie exceptionne